# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config2 import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Make a reference to the books.csv file path
csv_path = "Output_Data/weather_data.csv"

# Import the books.csv file as a DataFrame
weather_df = pd.read_csv(csv_path, encoding="utf-8")
weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Mataura,-46.1927,168.8643,19.80,49,53,3.05
1,1,Cloquet,46.7216,-92.4594,2.43,97,90,3.60
2,2,Abu Samrah,35.3029,37.1841,10.56,50,0,2.03
3,3,Lüderitz,-26.6481,15.1594,19.35,50,0,8.31
4,4,Buchanan,5.8808,-10.0467,25.30,86,63,0.77


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
Humidity = weather_df["Humidity"].astype(float)

In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Ideal weather conditions

- A max temperature lower than 80 degrees but higher than 70
- Wind speed less than 10 mph
- Zero cloudiness

In [6]:
ideal_weather = weather_df.loc[weather_df["Wind Speed"]<10]
ideal_weather


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Mataura,-46.1927,168.8643,19.80,49,53,3.05
1,1,Cloquet,46.7216,-92.4594,2.43,97,90,3.60
2,2,Abu Samrah,35.3029,37.1841,10.56,50,0,2.03
3,3,Lüderitz,-26.6481,15.1594,19.35,50,0,8.31
4,4,Buchanan,5.8808,-10.0467,25.30,86,63,0.77
...,...,...,...,...,...,...,...,...
538,538,Kangaatsiaq,68.3065,-53.4641,-11.19,91,88,4.53
539,539,Guanare,9.0418,-69.7421,24.62,69,72,2.01
540,540,Ansalta,42.6894,46.1178,0.57,73,2,1.77
541,541,Gayéri,12.6497,0.4931,26.60,19,7,2.84


In [7]:
ideal_weather = weather_df.loc[weather_df["Cloudiness"]==0]
ideal_weather.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Abu Samrah,35.3029,37.1841,10.56,50,0,2.03
3,3,Lüderitz,-26.6481,15.1594,19.35,50,0,8.31
12,12,Punta Arenas,-53.1500,-70.9167,10.06,57,0,5.14
16,16,Cape Town,-33.9258,18.4232,17.83,85,0,1.79
18,18,Coquimbo,-29.9533,-71.3436,12.85,87,0,2.06


# Cities - tempearture beween 70F (21.1 Celcius) to 80F (26.6 Celcius)

In [8]:
ideal_weather = weather_df.loc[(weather_df["Max Temp"]>21.1) & (weather_df["Max Temp"]<26.6)]
ideal_weather

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
4,4,Buchanan,5.8808,-10.0467,25.30,86,63,0.77
5,5,São Filipe,14.8961,-24.4956,24.47,76,24,3.54
6,6,Atuona,-9.8000,-139.0333,25.68,76,10,5.22
11,11,Iracoubo,5.4802,-53.2011,25.30,90,91,3.26
13,13,Tamiahua,21.2667,-97.4500,24.16,75,57,2.81
...,...,...,...,...,...,...,...,...
523,523,Esim,4.8699,-2.2405,25.55,87,60,1.31
525,525,Atarra,25.2833,80.5667,21.26,44,63,0.73
527,527,San Fernando del Valle de Catamarca,-28.4696,-65.7852,23.69,55,15,6.87
528,528,Grand Gaube,-20.0064,57.6608,25.21,72,1,1.34


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = ideal_weather

In [10]:
hotel_df["Hotel Name"]=""
hotel_df

C:\Users\Laurent\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
4,4,Buchanan,5.8808,-10.0467,25.30,86,63,0.77,
5,5,São Filipe,14.8961,-24.4956,24.47,76,24,3.54,
6,6,Atuona,-9.8000,-139.0333,25.68,76,10,5.22,
11,11,Iracoubo,5.4802,-53.2011,25.30,90,91,3.26,
13,13,Tamiahua,21.2667,-97.4500,24.16,75,57,2.81,
...,...,...,...,...,...,...,...,...,...
523,523,Esim,4.8699,-2.2405,25.55,87,60,1.31,
525,525,Atarra,25.2833,80.5667,21.26,44,63,0.73,
527,527,San Fernando del Valle de Catamarca,-28.4696,-65.7852,23.69,55,15,6.87,
528,528,Grand Gaube,-20.0064,57.6608,25.21,72,1,1.34,


# geocoordinates
target_coordinates = "-31.9523, 115.8613"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": gkey
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [23]:
hotel_df = hotel_df.astype({"Lat":float,"Lng":float})

In [31]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# geocoordinates
# target_coordinates = "-31.9523, 115.8613"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
#     "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {hotel_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {hotel_name} hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 4: -69.0.
Missing field/result... skipping.
------------
Retrieving Results for Index 5: -69.0.
Missing field/result... skipping.
------------
Retrieving Results for Index 6: -69.0.
Missing field/result... skipping.
------------
Retrieving Results for Index 11: -69.0.
Missing field/result... skipping.
------------
Retrieving Results for Index 13: -69.0.
Missing field/result... skipping.
------------
Retrieving Results for Index 15: -69.0.
Missing field/result... skipping.
------------
Retrieving Results for Index 34: -69.0.
Missing field/result... skipping.
------------
Retrieving Results for Index 45: -69.0.
Closest -69.0 hotel is Rancho Hobonil UADY.
------------
Retrieving Results for Index 47: -69.0.
Closest -69.0 hotel is Mansoor's Manzil.
------------
Retrieving Results for Index 50: -69.0.
Missing field/result... skipping.
------------
Retrieving Results for Index 51: -69.0.
Missing field/result... skipping.
------------
Retrieving Results for Index 

Missing field/result... skipping.
------------
Retrieving Results for Index 417: -69.0.
Missing field/result... skipping.
------------
Retrieving Results for Index 423: -69.0.
Missing field/result... skipping.
------------
Retrieving Results for Index 431: -69.0.
Missing field/result... skipping.
------------
Retrieving Results for Index 434: -69.0.
Missing field/result... skipping.
------------
Retrieving Results for Index 438: -69.0.
Closest -69.0 hotel is RANCHO ECOTURISTICO LOS TRES POTRILLOS.
------------
Retrieving Results for Index 442: -69.0.
Missing field/result... skipping.
------------
Retrieving Results for Index 458: -69.0.
Missing field/result... skipping.
------------
Retrieving Results for Index 464: -69.0.
Missing field/result... skipping.
------------
Retrieving Results for Index 465: -69.0.
Missing field/result... skipping.
------------
Retrieving Results for Index 467: -69.0.
Missing field/result... skipping.
------------
Retrieving Results for Index 468: -69.0.
Clo

In [32]:
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
4,4,Buchanan,5.0,-10.0,25.30,86,63,0.77,
5,5,São Filipe,14.0,-24.0,24.47,76,24,3.54,
6,6,Atuona,-9.0,-139.0,25.68,76,10,5.22,
11,11,Iracoubo,5.0,-53.0,25.30,90,91,3.26,
13,13,Tamiahua,21.0,-97.0,24.16,75,57,2.81,
...,...,...,...,...,...,...,...,...,...
523,523,Esim,4.0,-2.0,25.55,87,60,1.31,
525,525,Atarra,25.0,80.0,21.26,44,63,0.73,Yadav resident
527,527,San Fernando del Valle de Catamarca,-28.0,-65.0,23.69,55,15,6.87,
528,528,Grand Gaube,-20.0,57.0,25.21,72,1,1.34,


In [37]:
# NOTE: Do not change any of the code in this cell
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Add marker layer ontop of heat map


# Display figure
